In [ ]:
import pandas as pd
import boto3
import json

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY =                       config['AWS']['KEY']
SECRET =                    config['AWS']['SECRET']

DWH_CLUSTER_TYPE =          config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES =             config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE =             config['DWH']['DWH_NODE_TYPE']
DWH_CLUSTER_IDENTIFIER =    config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB =                    config['DWH']['DWH_DB']
DWH_DB_USER =               config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD =           config['DWH']['DWH_DB_PASSWORD']
DWH_PORT =                  config['DWH']['DWH_PORT']
DWH_IAM_ROLE_NAME =         config['DWH']['DWH_IAM_ROLE_NAME']

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [ ]:
ec2 = boto3.resource(
    'ec2',
    region_name='us-west-2',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
    )

s3 = boto3.resource(
    's3',
    region_name='us-west-2',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
    )

iam = boto3.client(
    'iam',
    region_name='us-west-2',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
    )

redshift = boto3.resource(
    'redshift',
    region_name='us-west-2',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
    )

In [ ]:
from botocore.exceptions import ClientError

try:
    print("1.1 creating a new IAM role")
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description='Allows Redshift clusters to call AWS services on your behalf.',
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
            'Effect': 'Allow',
            'Principal': {'Service': 'redshift.amazonaws.com'}}],
            'Version': '2012-10-17'}
        )
    )
except Exception as e:
    print(e)

print('1.2 Attaching Policy')
iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess')['ResponseMetadata']['HTTPStatusCode']

print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(
    RoleName=DWH_IAM_ROLE_NAME
)['Role']['Arn']

print(roleArn)

In [ ]:
try:
    response = redshift.create_cluster(
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,

        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ['ClusterIdentifier', 'NodeType', 'ClusterStatus', 'MasterUsername', 'DBName', 'Endpoint', 'NumberOfNodes', 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=['Key', 'Value'])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print('DWH_ENDPOINT :: ', DWH_ENDPOINT)
print('DWH_ROLE_ARN :: ', DWH_ROLE_ARN)

In [ ]:
try:
    vpc = ec2.Vpc(
        id=myClusterProps['VpcId']
    )
    defaultSg = list(vpc.security_groups.all())[0]
    print(deafultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

In [ ]:
%load_ext sql

In [ ]:
conn_string='postgresql://{}:{}@{}:{}/{}'.format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

In [ ]:
redshift.delete_cluster(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    SkipFinalClusterSnapshot=True
    )

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
iam.detach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
)
iam.delete_role(
    RoleName=DWH_IAM_ROLE_NAME
)